In [186]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

# set env
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)


from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()



data_root = '../data/_datasets/cluster_res'
# 分类数据
pwtc_f = 'protein_wt_cluster.tsv'
ddna_f = 'dsDNA_80_cluster.txt'
drna_f = 'dsRNA_80_cluster.txt'
sdna_f = 'ssDNA_80_cluster.txt'
srna_f = 'ssRNA_80_cluster.txt'
# 主要数据
# mainf = 'seq_dg_230620.txt'
mainf = 'seq_dg_v01.txt'
wtf = 'wt_v01.tsv'


mainf = os.path.join(data_root, mainf)
madf = pd.read_csv(mainf, sep='\t', low_memory=False)
print(len(madf))
# madf.head(3)

madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))

# seq_dg_230620.txt 是大文件;
# dataset_new 是更新后，各个核酸复合物的数据集文件;
# cluster_new 是聚类文件,wt 数据集直接将 uniprot+na连接表示一个复合物，
# mut 将 uniprot+protein_mutation+na 表示复合物。
# 单双链的na不同，单链为 na_index列,双链为 newnafea_na_job 列;

# mutation_type
# UniProt
# nucleic_acid_type_new
# double: newnafea_na_job
# single: na_index


wtdf = pd.read_csv( os.path.join(data_root, wtf), sep='\t' )
print(len(wtdf))
# wtdf.head(3)
# protein_sequence nucleotide_sequence dG complex_id

wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))

# seq2data = dict()
# uniq_dlist = list()
# for i in range(len(madf)):
#     row = madf.loc[i]
#     seq = row['protein_sequence'] + row['nucleotide_sequence']
#     if seq not in seq2data:
#         seq2data[seq] = 1
#         uniq_dlist.append(True)
#     else:
#         uniq_dlist.append(False)

# mask = pd.Series(uniq_dlist)
# madf = madf[mask]
# madf = madf.reset_index()
# print("after remove, len =", len(madf))

### check valid
# for i in tqdm(range(len(wtdf))):
#     row = wtdf.loc[i]
#     key = row['protein_sequence'] + row['nucleotide_sequence']
#     cid = row['complex_id']
#     tmp = madf[madf['complex_id'] == cid]
#     key2 = tmp['protein_sequence'].iloc[0] + tmp['nucleotide_sequence'].iloc[0]
#     if key != key2:
#         print(cid)
#     if len(tmp) != 1:
#         print(cid)

17199
4846


In [8]:
# 相同蛋白序列但是UniProt ID 不同：
# P0A6X3,A0A140NGK1,C1IFD2
# P24042,Q71TA8
# Q8DQG2,A0A0H2UPA7
# P0AGK8 C3SZN7

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))


In [187]:
def createMap(data_root, fpath, prefix, output, cls):
    df = pd.read_csv( os.path.join(data_root, fpath), sep='\t')
    cls = df['cluster_index'].values
    for i in range(len(df)):
        row = df.loc[i]
        keys = row['cluster_member'].split(',')
        for k in keys:
            output[k] = prefix + row['cluster_index']

cid2wt = dict()
for c in madf.loc[:, 'complex_id'].values:
    cid2wt[c] = False

### 创建mapping
## cluster_index  cluster_number  cluster_member
## UniProt -> class_name
## key_nucleic_acids -> class_name
prot_cls = None
unp2cls = dict()
createMap(data_root, pwtc_f, 'P', unp2cls, prot_cls)

ddna_cls, drna_cls, sdna_cls, srna_cls = None, None, None, None
ddna2cls, drna2cls, sdna2cls, srna2cls = dict(), dict(), dict(), dict()
createMap(data_root, ddna_f, 'DDNA', ddna2cls, ddna_cls)
createMap(data_root, drna_f, 'DRNA', drna2cls, drna_cls)
createMap(data_root, sdna_f, 'SDNA', sdna2cls, sdna_cls)
createMap(data_root, srna_f, 'SRNA', srna2cls, srna_cls)


### 处理wt数据
if 'pclass' not in wtdf.columns:
    wtdf.insert(len(wtdf.columns), 'pclass', 'None', allow_duplicates=False)
if 'nuclass' not in madf.columns:
    madf.insert(len(madf.columns), 'nuclass', 'None', allow_duplicates=False)
    
for i in range(len(wtdf)):
    row = wtdf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in wt data: ", i, row['complex_id'], row['UniProt'])
    else:
        wtdf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]
    cid2wt[row['complex_id']] = True
    #
    
    nc_type = row['nucleic_acid_type_new']
    # d_ncid = row['newnafea_na_job']
    # s_ncid = row['na_index']
    key_na = row['key_nucleic_acids']
    if nc_type == 'dsDNA':
        wtdf.loc[i, 'nuclass'] = ddna2cls[ key_na ]
    elif nc_type == 'dsRNA':
        wtdf.loc[i, 'nuclass'] = drna2cls[ key_na ]
    elif nc_type == 'ssDNA':
        wtdf.loc[i, 'nuclass'] = sdna2cls[ key_na ]
    elif nc_type == 'ssRNA':
        wtdf.loc[i, 'nuclass'] = srna2cls[ key_na ]
    else:
        print("is not valid dna/rna type")


### 处理主数据
if 'wt_complex' not in madf.columns:
    madf.insert(len(madf.columns), 'wt_complex', False, allow_duplicates=False)
if 'pclass' not in madf.columns:
    madf.insert(len(madf.columns), 'pclass', 'None', allow_duplicates=False)
if 'nuclass' not in madf.columns:
    madf.insert(len(madf.columns), 'nuclass', 'None', allow_duplicates=False)

for i in range(len(madf)):
    row = madf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in ma data: ", i, row['complex_id'], row['UniProt'])
    else:
        madf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]
    #
    madf.loc[i, 'wt_complex'] = cid2wt[ row['complex_id'] ]
    #
    nc_type = row['nucleic_acid_type_new']
    # d_ncid = row['newnafea_na_job']
    # s_ncid = row['na_index']
    key_na = row['key_nucleic_acids']
    if nc_type == 'dsDNA':
        madf.loc[i, 'nuclass'] = ddna2cls[ key_na ]
    elif nc_type == 'dsRNA':
        madf.loc[i, 'nuclass'] = drna2cls[ key_na ]
    elif nc_type == 'ssDNA':
        madf.loc[i, 'nuclass'] = sdna2cls[ key_na ]
    elif nc_type == 'ssRNA':
        madf.loc[i, 'nuclass'] = srna2cls[ key_na ]
    else:
        print("is not valid dna/rna type")

# mutation_type
# UniProt
# nucleic_acid_type_new
# double: newnafea_na_job
# single: na_index

madf['base_class'] = madf[['nuclass', 'pclass']].agg('-'.join, axis=1)
wtdf['base_class'] = wtdf[['nuclass', 'pclass']].agg('-'.join, axis=1)

print('done')

done


In [188]:
# gkey is str of list of str
def createGroup(gkey, indf, groups_by_gkey):
    for k, subdf in indf.groupby(gkey):
        cp = subdf.copy(deep=True)
        cp = cp.reset_index()
        print(k, len(cp))
        groups_by_gkey[k] = cp
#

# all_cls2ind = dict()
# all_cls_set = set(madf['base_class'])
# for i, c in enumerate(all_cls_set):
#     all_cls2ind[c] = i

# for k, subdf in madf.groupby(['pclass', 'nuclass']):
#     if len(subdf) > 1:
#         print(k, len(subdf))
groups_by_wt = dict()
for k, subdf in madf.groupby('wt_complex'):
    cp = subdf.copy(deep=True)
    cp = cp.reset_index()
    print(k, len(cp))
    groups_by_wt[k] = cp
    
wt_sub_df = groups_by_wt[True]
mut_sub_df = groups_by_wt[False]

all_cls_set = set(madf['base_class'].values)
wt_cls_set = set(wt_sub_df['base_class'].values)
mut_cls_set = set(mut_sub_df['base_class'].values)

# uniq_wt_cls_indices = set([all_cls2ind[c] for c in wt_cls_set])
# uniq_mut_cls_indices = set([all_cls2ind[c] for c in mut_cls_set])
# cls_indices_only_in_wt = uniq_wt_cls_indices - uniq_mut_cls_indices

# in WT but not in MUT
cls_only_in_wt = wt_cls_set - mut_cls_set

# len(all_cls_set) - len(mut_cls_set - wt_cls_set) - len(cls_only_in_wt)
### only in WT: 3207
### only in MUT: 207
### in Both: 808
### all: 4222


### map from base class to complex id
map_cls2cid = dict()
for k, subdf in madf.groupby('base_class'):
    map_cls2cid[k] = subdf['complex_id'].values
    
###

False 12356
True 4843


In [189]:

# na_type = row['nucleic_acid_type_new']
# if na_type == 'dsDNA':
#     df.loc[i, 'key_nucleic_acids'] = row['newnafea_na_job'].replace('new_DNA', 'Double_DNA')
# elif na_type == 'dsRNA':
#     df.loc[i, 'key_nucleic_acids'] = row['newnafea_na_job'].replace('new_RNA', 'Double_RNA')
# elif na_type == 'ssDNA':
#     df.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssDNA', 'Single_DNA_')
# elif na_type == 'ssRNA':
#     df.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssRNA', 'Single_RNA_')
def read2df(root, f):
    f = os.path.join(root, f)
    return pd.read_csv(f, sep='\t') # , low_memory=False

dir_sim = 'similarity'

sim_prot_f = 'protein_dist_230703.txt'
sim_ddna_f = 'dsdna_dist_230703.txt'  
sim_drna_f = 'dsrna_dist_230703.txt'
sim_sdna_f = 'ssdna_dist_230703.txt'  
sim_srna_f = 'ssrna_dist_230703.txt'

sim_dir = os.path.join(data_root, dir_sim)

sim_prot_df = read2df(sim_dir, sim_prot_f)
sim_ddna_df = read2df(sim_dir, sim_ddna_f)
sim_drna_df = read2df(sim_dir, sim_drna_f)
sim_sdna_df = read2df(sim_dir, sim_sdna_f)
sim_srna_df = read2df(sim_dir, sim_srna_f)

map_naType2simDf = {
    'dsDNA' : sim_ddna_df,
    'dsRNA' : sim_drna_df,
    'ssDNA' : sim_sdna_df,
    'ssRNA' : sim_srna_df,
}



def computeSimMat(df, sim_df, input_key, map_Key2Ind):
    key_set = set( df[input_key] )

    map_local = {nkey: lid for lid, nkey in enumerate(key_set)}
    for lid, nkey in enumerate(key_set):
        map_Key2Ind[nkey] = lid
        
    sim_mat = np.zeros([len(key_set), len(key_set)], dtype=np.float64)
    n_sum = 0

    query, target, fident = sim_df['query'], sim_df['target'], sim_df['fident']
    query_ind = query.apply(lambda x: map_local[x]).values
    target_ind = target.apply(lambda x: map_local[x]).values
    fident_vec = fident.values
    print(sim_mat.shape)
    print(len(query_ind), len(target_ind), len(fident_vec))
    sim_mat[query_ind, target_ind] = fident_vec

    print("{}, positive sim value = {}, sim mat shape = {}".format(input_key, n_sum, sim_mat.shape))
    return sim_mat


map_pKey2simMatInd = dict()
prot_sim_mat = computeSimMat(madf, sim_prot_df, 'protein_index', map_pKey2simMatInd)


# map_naType2naKey = dict()
map_naType2simMat = dict()
map_naKey2simMatInd = dict()
for na_type, sdf in madf.groupby('nucleic_acid_type_new'):
    map_naType2simMat[na_type] = computeSimMat(sdf, 
                                               map_naType2simDf[na_type], 
                                               'key_nucleic_acids', 
                                               map_naKey2simMatInd)
print("done")

(3069, 3069)
4601061 4601061 4601061
protein_index, positive sim value = 0, sim mat shape = (3069, 3069)
(1804, 1804)
13501 13501 13501
key_nucleic_acids, positive sim value = 0, sim mat shape = (1804, 1804)
(90, 90)
222 222 222
key_nucleic_acids, positive sim value = 0, sim mat shape = (90, 90)
(1602, 1602)
16556 16556 16556
key_nucleic_acids, positive sim value = 0, sim mat shape = (1602, 1602)
(909, 909)
12870 12870 12870
key_nucleic_acids, positive sim value = 0, sim mat shape = (909, 909)
done


In [205]:
### compare similarity

g_prot_cls2ind = dict()
g_ddna_cls2ind = dict()
g_drna_cls2ind = dict()
g_sdna_cls2ind = dict()
g_srna_cls2ind = dict()

def getMeanSim(sim_mat, map1, map2, cids_a, cids_b):
    '''
    # sim_mat: np array
    '''
    # cid -> protein_index -> sim_mat.index
    indices_a = np.array([ map2[map1[c]] for c in cids_a])
    indices_b = np.array([ map2[map1[c]] for c in cids_b])
    print("a: {} b: {}".format(indices_a, indices_b))
    
    sim = sim_mat[indices_a, indices_b] # (num_a, num_b)
    return sim.mean()
    

### input i and j are value from sim_mat
def computeDist(i, j):
    i += 1
    j += 1
    return i * j * (i + j) / (np.abs(i - j) + 1)


# ma_sim_f = 'seq_dg_230703.txt'
# ma_sim_f = os.path.join(data_root, ma_sim_f)
# ma_sim_df = pd.read_csv(ma_sim_f, sep='\t', low_memory=False)

map_cid2pKey = dict()
map_cid2naKey   = dict()
for i in range(len(madf)):
    row = madf.loc[i]
    map_cid2pKey[ row['complex_id'] ] = row['protein_index']
    map_cid2naKey[ row['complex_id'] ] = row['key_nucleic_acids']


# prot_sim_mat
# map_pKey2simMatInd
# map_naType2simMat
# map_naKey2simMatInd



def getDataByNcType(class_name, mapping):
    '''
    # mapping: nc type -> sim mat df
    '''
    if 'DDNACluster' in class_name:
        return 'dsDNA', mapping['dsDNA']
    elif 'DRNACluster' in class_name:
        return 'dsRNA', mapping['dsRNA']
    elif 'SDNACluster' in class_name:
        return 'ssDNA', mapping['ssDNA']
    elif 'SRNACluster' in class_name:
        return 'ssRNA', mapping['ssRNA']
    else:
        print("get base class name with a wrong NC class: {}".format(class_name))
        return None, None

###
for ci in all_cls_set:
    for cj in all_cls_set:
        cids_i = map_cls2cid[ci]
        cids_j = map_cls2cid[cj]
        # clu_i ==> prot_clu_i, nc_clu_i
        # clu_j ==> prot_clu_j, nc_clu_j
        nc_type_i, nc_sim_data_i = getDataByNcType(ci, map_naType2simMat)
        nc_type_j, nc_sim_data_j = getDataByNcType(cj, map_naType2simMat)
        
        mP_sim = getMeanSim(prot_sim_mat, map_cid2pKey, map_pKey2simMatInd, cids_i, cids_j)
        
        mN_dist = None
        if nc_type_i == nc_type_j:
            print(nc_type_i)
            mN_sim = getMeanSim(map_naType2simMat[nc_type_i], 
                                map_cid2naKey,
                                map_naKey2simMatInd,
                                cids_i, cids_j)
        else:
            # different type indicates largest distance 
            mN_dist = np.ones([len(cids_i), len(cids_j)], dtype=np.float64)

a: [2471] b: [2471]
dsDNA
a: [879] b: [879]
a: [2471] b: [1420]
a: [2471] b: [1531 1531]
a: [2471] b: [1339 1339]
a: [2471] b: [716 716]
a: [2471] b: [2290]
dsDNA
a: [879] b: [555]
a: [2471] b: [2070]
a: [2471] b: [1223]
a: [2471] b: [780 780]
dsDNA
a: [879] b: [224 224]
a: [2471] b: [2297 2297]
dsDNA
a: [879] b: [380 380]
a: [2471] b: [2048 2319 2022 2356  165  165]
dsDNA
a: [879] b: [666 666 666 666 666 666]
a: [2471] b: [ 207  207 1956 1956]
a: [2471] b: [459 459]
a: [2471] b: [2899 2785]
a: [2471] b: [1586]
a: [2471] b: [2764 2764]
a: [2471] b: [431 431]
a: [2471] b: [2742]
a: [2471] b: [2849]
a: [2471] b: [702]
a: [2471] b: [431 170]
a: [2471] b: [1339]
dsDNA
a: [879] b: [313]
a: [2471] b: [113]
dsDNA
a: [879] b: [1610]
a: [2471] b: [1667  592 2762 1667  592 2762]
a: [2471] b: [2886]
a: [2471] b: [247 597 597 247]
a: [2471] b: [2697 2697]
a: [2471] b: [2224 2224]
a: [2471] b: [2261 2261 2261 2261 2261 2261 2261 2261 2261 2261 2261 2261 2261 2261]
a: [2471] b: [2505 2505 2505]
a: [

a: [2471] b: [2886]
a: [2471] b: [2830]
a: [2471] b: [909 909]
dsDNA
a: [879] b: [1127 1127]
a: [2471] b: [2651 2651]
a: [2471] b: [ 518 1439  990 2735 2761 1606 2577 1326  941 1390 2218  609]
a: [2471] b: [2045 2045]
dsDNA
a: [879] b: [1247 1247]
a: [2471] b: [2381 2381 2381 2381 2381 2381 1599 1599 1599 1599 1599 1356 1356 1356
 1356 1356    0    0  618  618  618  618  618]
a: [2471] b: [2680]
a: [2471] b: [2667 2667 2667 2667 2667 2667 2667 2667 2667 2667 2667 2667]
dsDNA
a: [879] b: [ 271  867 1193 1193 1193 1261 1261 1261    8    8    8    8]
a: [2471] b: [2854 2854 2854 2854]
a: [2471] b: [1529]
dsDNA
a: [879] b: [825]
a: [2471] b: [558 558]
a: [2471] b: [2153 2153]
a: [2471] b: [2490 2490 2490 2490 2490 2490 2490 2490]
dsDNA
a: [879] b: [126 126 126 126 126 126 126 126]
a: [2471] b: [2650]
dsDNA
a: [879] b: [246]
a: [2471] b: [2532  130  458  458]
a: [2471] b: [585 585]
dsDNA
a: [879] b: [193 193]
a: [2471] b: [1311]
dsDNA
a: [879] b: [18]
a: [2471] b: [ 693  693 2572 2572]
a: [

a: [2471] b: [3068 3068 3068  136  136  136  136  136  372  372  372  372 2548 2548]
a: [2471] b: [ 696  766 1300 1171 1379]
a: [2471] b: [558 558]
a: [2471] b: [671 671 671]
a: [2471] b: [2070]
a: [2471] b: [2527 2527]
a: [2471] b: [2840 1095  809 3010 1535 3012  842 1623 2840 1095  809 3010 1535 3012
  842 1623]
dsDNA
a: [879] b: [848 848 848 848 848 848 848 848 848 848 848 848 848 848 848 848]
a: [2471] b: [1533 1533 1533 1533]
a: [2471] b: [2886]
a: [2471] b: [1533]
a: [2471] b: [ 285 1755]
a: [2471] b: [282 282]
a: [2471] b: [1059 1059 1059 1059]
a: [2471] b: [ 915 1950   98 1156]
dsDNA
a: [879] b: [1324 1324 1324 1324]
a: [2471] b: [947]
dsDNA
a: [879] b: [1637]
a: [2471] b: [2863]
a: [2471] b: [1994 1994]
a: [2471] b: [0 0 0]
dsDNA
a: [879] b: [1672 1672 1672]
a: [2471] b: [1225 1225 1225 1225]
a: [2471] b: [2697 2697 2681 1019 1551 1138 2697 2697 2697 2697]
a: [2471] b: [2680 2680]
dsDNA
a: [879] b: [1246 1246]
a: [2471] b: [2048]
a: [2471] b: [366 366]
a: [2471] b: [1120 1120 

a: [1420] b: [1745]
a: [1420] b: [2817]
ssDNA
a: [134] b: [746]
a: [1420] b: [1974 2493 2994  507 1214 1159 1329 2654 1974 2493 2994  507 1214 1159
 1329 2654]
ssDNA
a: [134] b: [1479 1479 1479 1479 1479 1479 1479 1479 1479 1479 1479 1479 1479 1479
 1479 1479]
a: [1420] b: [397]
a: [1420] b: [399 399]
a: [1420] b: [2678]
a: [1420] b: [780 780]
a: [1420] b: [2860 2811  701 2610 2526 2860 2811  701 2610 2526]
a: [1420] b: [1988 2741 2711  761 3049  434 3024]
ssDNA
a: [134] b: [44 44 44 44 44 44 44]
a: [1420] b: [2351 2351]
a: [1420] b: [684]
a: [1420] b: [28 28 28 28]
ssDNA
a: [134] b: [878 878 878 878]
a: [1420] b: [995 995]
ssDNA
a: [134] b: [602 602]
a: [1420] b: [2688]
ssDNA
a: [134] b: [371]
a: [1420] b: [  80 1918]
a: [1420] b: [1721 1721]
ssDNA
a: [134] b: [610 610]
a: [1420] b: [658 658 658 658]
ssDNA
a: [134] b: [1517 1517 1517 1517]
a: [1420] b: [1619]
ssDNA
a: [134] b: [954]
a: [1420] b: [1895]
ssDNA
a: [134] b: [453]
a: [1420] b: [1248 1248 1373 2852 2011 2934 2934 2306  529 

a: [1420] b: [596 596 596]
a: [1420] b: [ 132 1499 1499 1499 1499 1499 1499 1499  132]
a: [1420] b: [275 275]
a: [1420] b: [2955 2955]
ssDNA
a: [134] b: [1096 1096]
a: [1420] b: [2127 2127 2127]
ssDNA
a: [134] b: [1124 1124 1124]
a: [1420] b: [1743 1743]
ssDNA
a: [134] b: [1504 1504]
a: [1420] b: [556 556]
a: [1420] b: [2505 2505]
ssDNA
a: [134] b: [264 264]
a: [1420] b: [349 349]
a: [1420] b: [334 334]
a: [1420] b: [2505 2505 2505]
ssDNA
a: [134] b: [779 779 779]
a: [1420] b: [1524 1524]
a: [1420] b: [ 769  786 3059 3059  246  246  446  446  446  769  786 3059 3059  246
  246  446  446  446]
ssDNA
a: [134] b: [386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386]
a: [1420] b: [1000 1000 1000 1000]
a: [1420] b: [648 648 648 648 648 648 648 648]
a: [1420] b: [1429 1429 2152 2152 1429 1429 1429 1429 2152 2152 2152 2152]
a: [1420] b: [2957  957  957]
a: [1420] b: [1652  158 2307 1652]
a: [1420] b: [2048]
ssDNA
a: [134] b: [1408]
a: [1420] b: [1745 1745 1745 1745]
a: [14

a: [1420] b: [240 240]
a: [1420] b: [2176 2176]
a: [1420] b: [2742 2742]
ssDNA
a: [134] b: [849 849]
a: [1420] b: [747 747]
ssDNA
a: [134] b: [86 86]
a: [1420] b: [399 399]
a: [1420] b: [2997]
ssDNA
a: [134] b: [972]
a: [1420] b: [1598 1598]
a: [1420] b: [247]
ssDNA
a: [134] b: [1527]
a: [1420] b: [849 849 849 849 849 849 849 849 849 849 849]
ssDNA
a: [134] b: [1206 1206 1206 1206 1206 1206 1206 1206 1206 1206 1206]
a: [1420] b: [2460 2460]
ssDNA
a: [134] b: [641 641]
a: [1420] b: [1334  840 1334  840]
ssDNA
a: [134] b: [945 945 945 945]
a: [1420] b: [1471 1246]
ssDNA
a: [134] b: [873 873]
a: [1420] b: [2955 2955]
a: [1420] b: [747]
a: [1420] b: [1655   37   37   37   37   37   37   37   37   37   37   37   37   37
   37   37   37   37   37 2878 2878 2878 2878 2878 2878 2878 2878 2878
 2878 2878 2878 2878 2878 2878 2878 2878 2878 1655 1655 1655 1655 1655
 1655 1655]
a: [1420] b: [2927 2927]
a: [1420] b: [2697 2697]
a: [1420] b: [2886]
ssDNA
a: [134] b: [1116]
a: [1420] b: [558 558]
a: 

a: [134] b: [1572]
a: [1420] b: [2956]
a: [1420] b: [2955]
a: [1420] b: [1931]
a: [1420] b: [849 849 849 849]
ssDNA
a: [134] b: [185 185 185 185]
a: [1420] b: [489 489 489 489]
a: [1420] b: [3026 3026 3026]
a: [1420] b: [ 318  318 1720 1720 2913 2913 2557 2557]
ssDNA
a: [134] b: [1462 1462 1462 1462 1462 1462 1462 1462]
a: [1420] b: [2696 2696]
a: [1420] b: [2045 2045]
a: [1420] b: [3051  620 1775 3051  620 1775]
a: [1420] b: [947 947]
ssDNA
a: [134] b: [1093 1093]
a: [1420] b: [716]
a: [1420] b: [134 134 134 134]
a: [1420] b: [2697 2697]
a: [1420] b: [2362 2555 2362 2555]
ssDNA
a: [134] b: [805 805 805 805]
a: [1420] b: [ 811  811  811  811  852  852  852  852  943  943  943  943 2734 2734
 2734 2734 1077 1077 1077 1077 2893 2893]
ssDNA
a: [134] b: [138 138 138 138 138 138 138 138 138 138 138 138 138 138 138 138 138 138
 138 138 138 138]
a: [1420] b: [2045 2045 2045 2945 2045]
a: [1420] b: [467]
ssDNA
a: [134] b: [1073]
a: [1420] b: [2207 2207]
a: [1420] b: [1168]
a: [1420] b: [2157 2

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (2,) (6,) 

In [247]:
arr = np.empty((0), np.int32)
arr = np.append(arr, np.array([10,20,30]), axis=0)
arr

array([10, 20, 30])

In [235]:
series_pMids = madf['protein_index'].apply(lambda x: map_pKey2simMatInd[x])
series_pMids
new_series = pd.Series(series_pMids.values , index=madf['complex_id'])
new_series

complex_id
0        1531
1        2505
2        2505
3        1126
4        1126
         ... 
17256      23
17257    1150
17258    1150
17259    1499
17260    1499
Length: 17199, dtype: int64

In [239]:
np.where(np.isnan(new_series).values)

(array([], dtype=int64),)

In [242]:
a = 'xxx'
b = 'ttt'
m = dict()
m[a, b] = 1.0
m[b, a] = 22.

for k, v in m.items():
    k1, k2 = k
    print(k1, k2)


xxx ttt
ttt xxx


In [215]:
fnpz = os.path.join(data_root, 'output_v01.npz')
npz_dict = np.load(fnpz, allow_pickle=True)
map_cls_pair2dist = npz_dict['map_cls_pair2dist'].item()

dict

In [217]:
map_cls_pair2dist = npz_dict['map_cls_pair2dist'].item()

In [248]:
x = np.ones([10, 10])
np.mean(x, axis=1).shape

(10,)

In [249]:
cls_dist_f = os.path.join(data_root, 'dist_output_v01.npz')
if os.path.exists(cls_dist_f):
  cls_dist_npz_data = np.load(cls_dist_f, allow_pickle=True)
  mDist_cls = cls_dist_npz_data['mDist_cls']
  mat_cls_dist = cls_dist_npz_data['mat_cls_dist']
  print('load existed dist file from: {}'.format(cls_dist_f))

load existed dist file from: ../data/_datasets/cluster_res/dist_output_v01.npz


In [363]:
data_root = '../data/_datasets/cluster_res/'
main_f = 'seq_dg_v02.txt'


main_f = os.path.join(data_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

def func(df):
  map_expId2uniq = dict()
  map_expId2uniqExpId = dict()
  for k, subdf in df.groupby('key_complex'):
    eids = subdf['exp_id'].values
    l = len(eids)
    for eid in eids:
      map_expId2uniq[eid] = True if l == 1 else False
      map_expId2uniqExpId[eid] = eids[0]
    
  return map_expId2uniq, map_expId2uniqExpId

map_expId2uniq, map_expId2uniqExpId = func(main_df)

f = os.path.join(data_root, 'test_set_v01.npz')
data = np.load(f, allow_pickle=True)

eids1 = data['ddna']
eids2 = data['drna']
eids3 = data['sdna']
eids4 = data['srna']

all_test_ids = np.concatenate([eids1, eids2, eids3, eids4])

targets = np.empty((0), dtype=np.int64)
# targets = []
for it in [eids1, eids2, eids3, eids4]:
    # eids = np.empty((0), dtype=np.int64)
    
    
    target = np.array(list(set([map_expId2uniqExpId[x] for x in it])))
    print(target.shape)
    targets = np.append(targets, target, axis=0)
    # targets.append(target)
    
    arr_uniq = np.array([x for x in it if map_expId2uniq[x]])
    arr_uu = np.array([x for x in it if not map_expId2uniq[x]])
    
#     print(len(it), len(arr_uniq), len(arr_uu))

(860,)
(50,)
(786,)
(558,)


In [406]:
import sys
np.set_printoptions(threshold=sys.maxsize)
data_root = '../data/_datasets/cluster_res/'
main_f = 'seq_dg_v02_1.txt'

main_f = os.path.join(data_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

np.unique(main_df['base_class'].values)[1222]

'DDNACluster_799-PCluster_723'

In [416]:
data_root = '../data/tmp_data/val/'
test_f = 'test.csv'
test_f = os.path.join(data_root, test_f)
test_df = pd.read_csv(test_f, sep='\t', low_memory=False)
print(len(test_df))
test_df['avg_dist_to_others'].values.min()

2281


2.6011355571327184

In [414]:
data_root = '../data/tmp_data/train/'
test_f = 'train.csv'
test_f = os.path.join(data_root, test_f)
test_df = pd.read_csv(test_f, sep='\t', low_memory=False)
test_df['avg_dist_to_others'].values.min()

3.343992725242154

In [408]:
a = {'x':1, 'y':2}
b = {'xx':1, 'xy':2}
def f(a,b):
    a.update(b)
    c = 1+1
f(a,b)
b,a

({'xx': 1, 'xy': 2}, {'x': 1, 'y': 2, 'xx': 1, 'xy': 2})

In [411]:
np.array(list(a.keys()))

array(['x', 'y', 'xx', 'xy'], dtype='<U2')